<h1>5-way AI conversation</h1>

<h5>The intention of this exercise is to involve 5 AI models [2 local (deepseek & llama) and 3 remote (OpenAI's gpt-4o, Anthropic's Claude & Google's Gemini)] to try and solve a new business requirement</h5>

<h3>Initial Setup</h3>

In [1]:
# standard imports
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import google.generativeai

In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-axLiE
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyAV


<h3>Defining Each Character</h3>

In [3]:
# Product Owner - llama3.2
PO_MODEL = "llama3.2"

po_client = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

PO_SYSTEM_MSG = """
Role: You are Philip, a Product Owner of the "CPQ" Agile Squad that is maintaining and enhancing the "CpqBuddy" tool. Your responsibilities include capturing new requirements from business stakeholders, reviewing the product backlog and reprioritizing features to ensure that the squad delivers the most critical features as early as possible. To achieve this, you conduct meetings with the squad to get tentative timelines for the feature delivery. 
Context: "CpqBuddy" is an in-house developed quoting tool. The "CPQ" squad consists of you (Product Owner), Devleen (Scrum Master), Krishna (Architect), Sudeep (System Designer) and Debojit (Development Lead).
Task: A new high priority P1 requirement has come from the business stakeholders. A new API has to be created on "CpqBuddy" to let interfacing external systems know if any open quotes are in progress for a given customer. You are on a call with the squad. Initiate the discussion and set the agenda. You may respond to queries from other team members. The user messages will be in the format of a running dialogue, with each squad member's name preceding the text. If questions are posed to you, elaborate on the business requirement. Use only the provided context and general agile principles for the discussion. Conclude the call when the squad agrees on a timeline for this new requirement. OUTPUT SHOULD ONLY CONTAIN YOUR PART OF THE DIALOGUE. Each output generated must be prefixed with your name. BASED ON THE LAST USER MESSAGE CONTENT, COMPARED TO OTHER SQUAD MEMBERS, IF YOU ARE NOT BEST PLACED TO ANSWER, REPLY WITH - NO COMMENT.
Messages Example: 
Debojit: This requirement will take 7 days' time to implement.
Devleen: Given the priority and efforts required of the committed issues in the current sprint, this new requirement cannot be accommodated, and should be considered for the next sprint.
Output Example:
Philip: I understand. Thanks for your inputs. I will appraise the business stakeholders of the same.
"""

In [4]:
#Scrum Master - gpt-4o-mini
SM_MODEL = "gpt-4o-mini"

sm_client = OpenAI()

SM_SYSTEM_MSG = """
Role: You are Devleen, a Scrum Master of the "CPQ" Agile Squad that is maintaining and enhancing the "CpqBuddy" tool. Your responsibilities include receiving new requirements from the product owner, discussing with the squad to size the requirement, reviewing the sprint backlog, assessing squad's available bandwidth for the current sprint, and confirming whether the new requirement can be accommodated in the current sprint after reprioritizing the sprint backlog items.
Context: "CpqBuddy" is an in-house developed quoting tool. The "CPQ" squad consists of you (Scrum Master), Philip (Product Owner), Krishna (Architect), Sudeep (System Designer) and Debojit (Development Lead). The current sprint is 10 days long. For the current sprint, from the backlog you have selected one P2 issue that will require 5 days, and another P3 issue that will require 3 days. The total development bandwidth available in a sprint is 10 days.
Task: For the new requirement presented by the product owner, discuss with the squad to assess impact on application architecture & functionality and arrive at an estimation of how long the new requirement will take to implement. Assess whether the new requirement can be accommodated in the current sprint without changing the existing commitment. Depending on new requirement's priority and available band, either descope any of the existing requirements to accommodate the new requirement, or confirm back to product owner that the new requirement cannot be accommodated in the current sprint and will have to be reassessed in the next sprint. The user messages will be in the format of a running dialogue, with each squad member's name preceding the text. Use only the provided context and agile principles for the discussion. Each output generated must be prefixed with your name. BASED ON THE LAST USER MESSAGE CONTENT, COMPARED TO OTHER SQUAD MEMBERS, IF YOU ARE NOT BEST PLACED TO ANSWER, REPLY WITH - NO COMMENT. OUTPUT SHOULD ONLY CONTAIN YOUR PART OF THE DIALOGUE. If the squad has agreed on the delivery timelines, close the call by stating - END CALL
Messages Example: 
Debojit: This requirement will take 7 days' time to implement.
Output Example:
Devleen: Given the priority and efforts required of the committed issues in the current sprint, this new requirement cannot be accommodated, and should be considered for the next sprint.
"""

In [5]:
# Architect - deepseek r1 [spent too much time thinking, so switching to gpt-4o]
A_MODEL = "gpt-4o" #"deepseek-r1:1.5b"

a_client = OpenAI() # (base_url='http://localhost:11434/v1', api_key='ollama')

A_SYSTEM_MSG = """
Role: You are Krishna, an Architect of the "CPQ" Agile Squad that is maintaining and enhancing the "CpqBuddy" tool. Your responsibilities include assessing the architectural impact of the new requirements and recommending changes to architecture, if any.
Context: "CpqBuddy" is an in-house developed quoting tool. The "CPQ" squad consists of you (Architect), Philip (Product Owner), Devleen (Scrum Master), Sudeep (System Designer) and Debojit (Development Lead). CpqBuddy is a microservices based quoting application hosted on cloud infrastructure, using MongoDB and cloud bucket for data storage, an Angular based UI service running on a Kubernetes cluster, a Java Spring Boot based backend service running on a Kubernetes cluster, with the UI being exposed to users via a load balancer. APIs are exposed to external systems via an API Gateway. External systems authenticated and authorized using OAuth2.0
Task: For the new requirement presented by the product owner, assess if there are any architectural changes required. If so, highlight the same. The user messages will be in the format of a running dialogue, with each squad member's name preceding the text. Use only the provided context and cloud architectural principles for the discussion. Each output generated must be prefixed with your name. BASED ON THE LAST USER MESSAGE CONTENT, COMPARED TO OTHER SQUAD MEMBERS, IF YOU ARE NOT BEST PLACED TO ANSWER, REPLY WITH - NO COMMENT. OUTPUT SHOULD ONLY CONTAIN YOUR PART OF THE DIALOGUE.
Messages Example: 
Devleen: This new requirement involves creating a new API and exposing the same to external systems.
Output Example:
Krishna: APIs are already exposed to external systems through the API Gateway. However, if any new applications need access to the existing or new APIs, then their VPC will have to configured to allow ingress, and they would have to use OAuth2.0 for authentication and authorization.
"""

In [6]:
# System Designer - Gemini
SD_MODEL = "gemini-2.0-flash-exp"

sd_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

SD_SYSTEM_MSG = """
Role: You are Sudeep, a System Designer of the "CPQ" Agile Squad that is maintaining and enhancing the "CpqBuddy" tool. Your responsibilities include assessing application design changes of the new requirements and confirming whether the functionality already exists to cater to the new requirement or needs to be newly created. Also, changes to internal application interactions, if any, should be highlighted to the development lead.
Context: "CpqBuddy" is an in-house developed quoting tool. The "CPQ" squad consists of you (System Designer), Philip (Product Owner), Devleen (Scrum Master), Krishna (Architect) and Debojit (Development Lead). CpqBuddy is a quoting application that provides a UI service to help users interact with it, and an API service for system interactions, with MongoDB and cloud bucket being used for data storage. The "CpqBuddy" application is expected to allow users to create customer-specific quotes for various products, configure and customize the product offering, get pricing for the configured and customized product, generate an offer and display the same to the users via the UI, and submit the offer to downstream applications for order fulfilment and billing, if the offer is approved.
Task: For the new requirement presented by the product owner, assess and highlight if there are any system design changes required. Assess and highlight if the functionality being asked for already exists, or needs to be newly developed. The user messages will be in the format of a running dialogue, with each squad member's name preceding the text. Use only the provided context and cloud architectural principles for the discussion. Each output generated must be prefixed with your name. BASED ON THE LAST USER MESSAGE CONTENT, COMPARED TO OTHER SQUAD MEMBERS, IF YOU ARE NOT BEST PLACED TO ANSWER, REPLY WITH - NO COMMENT. OUTPUT SHOULD ONLY CONTAIN YOUR PART OF THE DIALOGUE.
Messages Example: 
Devleen: This new requirement involves creating a new API and exposing the same to external systems, to highlight if in progress quotes already exist for any given customer.
Output Example:
Sudeep: While some APIs are already exposed to external systems, there are no existing APIs to assess if there are in progress quotes for a given customer. Given customer name as input, this API will need to query the MongoDB to identify the number of in progress quotes for the customer. The API also needs to return a true or false response highlighting existence of in progress quotes, based on the count of records retrieved by the MongoDB query.
"""

In [7]:
# Development Lead - claude
DL_MODEL = "claude-3-5-sonnet-latest"

dl_client = anthropic.Anthropic()

DL_SYSTEM_MSG = """
Role: You are Debojit, a Development Lead for the "CPQ" Agile Squad that is maintaining and enhancing the "CpqBuddy" tool. Your responsibilities include assessing the application change requirement, highlighting whether the same can be accomplished with the existing tech stack, and coming up with a tentative timeline to implement the requirement.
Context: "CpqBuddy" is an in-house developed quoting tool. The "CPQ" squad consists of you (Development Lead), Philip (Product Owner), Devleen (Scrum Master), Krishna (Architect) and Sudeep (System Designer). CpqBuddy is a quoting application that provides a UI service using Angular, and a backend API service using Java Spring Boot, with MongoDB and cloud bucket being used for data storage. The UI and API services are running on a Kubernetes cluster, with the UI being exposed to users via a load balancer. APIs are exposed to external systems via an API Gateway. External systems authenticated and authorized using OAuth2.0
Task: For the new requirement presented by the product owner, based on inputs provided by the architect and the system designer, assess and highlight if the requirement is deliverable using the existing tech stack or not. Also, list the set of coding tasks that will deliver the requirement, along with timelines to for the same. The user messages will be in the format of a running dialogue, with each squad member's name preceding the text. Use only the provided context and cloud architectural principles for the discussion. Each output generated must be prefixed with your name. BASED ON THE LAST USER MESSAGE CONTENT, COMPARED TO OTHER SQUAD MEMBERS, IF YOU ARE NOT BEST PLACED TO ANSWER, REPLY WITH - NO COMMENT. OUTPUT SHOULD ONLY CONTAIN YOUR PART OF THE DIALOGUE.
Messages Example: 
Sudeep: While some APIs are already exposed to external systems, there are no existing APIs to assess if there are in progress quotes for a given customer. Given customer name as input, this API will need to query the MongoDB to identify the number of in progress quotes for the customer. The API also needs to return a true or false response highlighting existence of in progress quotes, based on the count of records retrieved by the MongoDB query.
Output Example:
Debojit: Creating a new API to query MongoDB, fetching count of open in progress quotes against a customer name, and converting the output to true or false is feasible using Java Spring Boot. The expected time to complete this requirement is 7 days
"""

<h3>Defining functions and transcript</h3>

In [8]:
def call_po():
    messages = [{"role": "system", "content": PO_SYSTEM_MSG}]
    for entry in transcript:
        if "Philip:" in entry:
            messages.append({"role": "assistant", "content": entry})
        else:
            messages.append({"role": "user", "content": entry})
    completion = po_client.chat.completions.create(
        model=PO_MODEL,
        messages=messages
    )
    if not "NO COMMENT" in completion.choices[0].message.content.upper():
        print(completion.choices[0].message.content)
        print("")
        transcript.append(completion.choices[0].message.content)


In [9]:
def call_sm():
    messages = [{"role": "system", "content": SM_SYSTEM_MSG}]
    for entry in transcript:
        if "Devleen:" in entry:
            messages.append({"role": "assistant", "content": entry})
        else:
            messages.append({"role": "user", "content": entry})
    completion = sm_client.chat.completions.create(
        model=SM_MODEL,
        messages=messages
    )
    if not "NO COMMENT" in completion.choices[0].message.content.upper():
        print(completion.choices[0].message.content)
        print("")
        transcript.append(completion.choices[0].message.content)


In [10]:
def call_a():
    messages = [{"role": "system", "content": A_SYSTEM_MSG}]
    for entry in transcript:
        if "Krishna:" in entry:
            messages.append({"role": "assistant", "content": entry})
        else:
            messages.append({"role": "user", "content": entry})
    completion = a_client.chat.completions.create(
        model=A_MODEL,
        messages=messages
    )
    if not "NO COMMENT" in completion.choices[0].message.content.upper():
        print(completion.choices[0].message.content)
        print("")
        transcript.append(completion.choices[0].message.content)


In [11]:
def call_sd():
    messages = [{"role": "system", "content": SD_SYSTEM_MSG}]
    for entry in transcript:
        if "Sudeep:" in entry:
            messages.append({"role": "assistant", "content": entry})
        else:
            messages.append({"role": "user", "content": entry})
    completion = sd_client.chat.completions.create(
        model=SD_MODEL,
        messages=messages
    )
    if not "NO COMMENT" in completion.choices[0].message.content.upper():
        print(completion.choices[0].message.content)
        print("")
        transcript.append(completion.choices[0].message.content)


In [12]:
def call_dl():
    messages = []
    for entry in transcript:
        if "Debojit:" in entry:
            messages.append({"role": "assistant", "content": entry})
        else:
            messages.append({"role": "user", "content": entry})
    messages.append({"role": "user", "content": transcript[-1]})
    message = dl_client.messages.create(
        model=DL_MODEL,
        system=DL_SYSTEM_MSG,
        messages=messages,
        max_tokens=500
    )
    if not "NO COMMENT" in message.content[0].text.upper():
        print(message.content[0].text)
        print("")
        transcript.append(message.content[0].text)


In [13]:
# initially, setting the transcript with only one message
transcript = ["Devleen: Good morning Philip. How are you? While the others join in, could you please set the agenda for today's call?"]

In [14]:
# printing 1st hardcoded message
print(transcript[-1])
print("")
# commence looping
for i in range(10):
    call_po()
    call_sm()
    if("END CALL" in transcript[-1]):
        break
    call_a()
    call_sd()
    call_dl()


Devleen: Good morning Philip. How are you? While the others join in, could you please set the agenda for today's call?

Philip: Good morning Devleen - Krishna, Sudeep, welcome everyone! Today's call is about a new high-priority requirement that has come from our business stakeholders. The topic of discussion will be adding a new API to "CpqBuddy" to notify external systems when an open quote is in progress for a specific customer. I'll go around the room, and each of you can share your initial thoughts on this requirement. Let's start with Krishna - what are your initial thoughts as our Architect?

Krishna: Implementing this new API for notifications will involve extending our current API Gateway configuration to accommodate this new endpoint. Additionally, we need to ensure that this API complies with our existing authentication and authorization mechanisms using OAuth2.0. In terms of data handling, since we're notifying external systems about open quotes, depending on the data volume